In [126]:
import socket
import binascii
import time
import struct

## Step 1: Build a DNS Query

In [127]:
# DNS Header

# Only working on ID, QR, Opcode, TC, RD, and QDCount (all other fields get value of 0)
# ID = 1010101000000000
# QR = 0
# Opcode = 0000
# TC = 0
# RD = 1
# QDCount = 0000000000000001

dnsHeader = "AA0001000001000000000000"

In [128]:
def input_padding(s):
    return '0x' + s[2:].zfill(2)


In [129]:
def dnsURLParser(user_input):
    host = ""
    tld = ""
    index_start = 0

    # Parse out hostname
    for element in range(0, len(user_input)):
        if user_input[element] == ".":
            index_start = element
            break
        else:
            host = host + user_input[element]

    # Parse out TLD
    for element in range(index_start+1, len(user_input)):
        tld = tld + user_input[element]


    hex_host = binascii.hexlify(host.encode("utf-8"))
    hex_tld = binascii.hexlify(tld.encode("utf-8"))

    len_host = input_padding(str(hex(len(host))))
    len_tld = input_padding(str(hex(len(tld))))

    query = len_host[2:] + hex_host.decode("utf-8") + len_tld[2:] + hex_tld.decode("utf-8")
    query = query.upper()
    print(query)

    return query

In [130]:
query = dnsURLParser("facebook.com")

0866616365626F6F6B03636F6D


In [131]:
# DNS Question
 
# QNAME = 03 74 6D 7A 03 63 6F 6D
# QTYPE (want A record) = 0000000000000001
# QCLASS = 0000000000000001

# to_hex(tmz) = 03 74 6D 7A 03 63 6F 6D

dnsQuestionBeta = query + "000001" + "0001"
jeff = len(dnsQuestionBeta)
#dnsQuestion = "03746D7A03636F6D0000010001"

In [132]:
dnsQuery = dnsHeader + dnsQuestionBeta
#print(dnsQuery)


## Step 2: UDP Socket Calls

In [133]:
# Iranian DNS: 91.245.229.1

# Create client-side socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# List out server details
server_addr = ("91.245.229.1", 53)

# Begin DNS Query and measure RTT
start_time = time.time()
client_socket.sendto(binascii.unhexlify(dnsQuery), server_addr)
iranian_requested_data, _ = client_socket.recvfrom(4096)

#print("Iranian DNS RTT: " + str(time.time() - start_time))
decoded_iranian_data = binascii.hexlify(iranian_requested_data).decode("utf-8")

KeyboardInterrupt: 

In [134]:
# American DNS: 169.237.229.88

# Create client-side socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# List out server details
server_addr = ("169.237.229.88", 53)

# Begin DNS Query and measure RTT
start_time = time.time()
client_socket.sendto(binascii.unhexlify(dnsQuery), server_addr)
american_requested_data, _ = client_socket.recvfrom(4096)

#print("American DNS RTT: " + str(time.time() - start_time))
decoded_american_data = binascii.hexlify(american_requested_data).decode("utf-8")



In [ ]:
# Canadian DNS: 136.159.85.15

# Create client-side socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# List out server details
server_addr = ("136.159.85.15", 53)

# Begin DNS Query and measure RTT
start_time = time.time()
client_socket.sendto(binascii.unhexlify(dnsQuery), server_addr)
canadian_requested_data, _ = client_socket.recvfrom(4096)

#print("Canadian DNS RTT: " + str(time.time() - start_time))
decoded_canadian_data = binascii.hexlify(canadian_requested_data).decode("utf-8")

## Step 3: Unpacking Header

In [135]:
def parse_header(decoded_data):
    id = ""
    flags = ""
    questions = ""
    answer_rr = ""
    authority_rr = ""
    additional_rr = ""


    for elementID in range(0, 4):
        id = id + decoded_data[elementID]
    
    for elementFlags in range(4, 8):
        flags = flags + decoded_data[elementFlags]

    for elementQuestions in range(8, 12):
        questions = questions + decoded_data[elementQuestions]
    
    for elementAnswerRR in range(12, 16):
        answer_rr = answer_rr + decoded_data[elementAnswerRR]
    
    for elementAuthorityRR in range(16, 20):
        authority_rr = authority_rr + decoded_data[elementAuthorityRR]
    
    for elementAdditionalRR in range(20, 24):
        additional_rr = additional_rr + decoded_data[elementAdditionalRR]
    
    flags_breakdown = bin(int(flags, 16))[2:].zfill(8)

    qr = ""
    opcode = ""
    aa = ""
    tc = ""
    rd = ""
    ra = ""
    z = ""
    rcode = ""

    for elementQR in range(0, 1):
        qr = qr + flags_breakdown[elementQR]
    #print("QR: ", qr)
    
    for elementOPCODE in range(1, 5):
        opcode = opcode + flags_breakdown[elementOPCODE]
    #print("Opcode: ", opcode)

    for elementAA in range(5, 6):
        aa = aa + flags_breakdown[elementAA]
    #print("AA: ", aa)

    for elementTC in range(6, 7):
        tc = tc + flags_breakdown[elementTC]
    #print("TC: ", tc)

    for elementRD in range(7, 8):
        rd = rd + flags_breakdown[elementRD]
    #print("RD: ", rd)

    for elementRA in range(8, 9):
        ra = ra + flags_breakdown[elementRA]
    #print("RA: ", ra)
    
    for elementZ in range(9, 12):
        z = z + flags_breakdown[elementZ]
    #print("Z: ", z)
    
    for elementRCODE in range(12, 16):
        rcode = rcode + flags_breakdown[elementRCODE]
    #print("Rcode: ", rcode)
    


    
    print("ID: ", bin(int(id, 16))[2:].zfill(16))
    print("Flags: ", bin(int(flags, 16))[2:].zfill(16))
    print("Flags Breakdown: ")
    print("\tQR: ", qr)
    print("\tOpcode: ", opcode)
    print("\tAA: ", aa)
    print("\tTC: ", tc)
    print("\tRD: ", rd)
    print("\tRA: ", ra)
    print("\tZ (Reserved): ", z)
    print("\tRcode: ", rcode)
    print("Number of entries in question section: ", int(questions, 16))
    print("Number of resource records in the answer section: ", int(answer_rr, 16))
    print("Number of name server resource records in the authority records section: ", int(authority_rr, 16))
    print("Number of resource records in the additional records section: ", int(additional_rr, 16))

    return int(answer_rr, 16)

In [ ]:
iranian_answer_RR = parse_header(decoded_iranian_data)

ID:  1010101000000000
Flags:  1000000110000000
Flags Breakdown: 
	QR:  1
	Opcode:  0000
	AA:  0
	TC:  0
	RD:  1
	RA:  1
	Z (Reserved):  000
	Rcode:  0000
Number of entries in question section:  1
Number of resource records in the answer section:  1
Number of name server resource records in the authority records section:  0
Number of resource records in the additional records section:  0


In [136]:
american_answer_RR = parse_header(decoded_american_data)

ID:  1010101000000000
Flags:  1000000110000000
Flags Breakdown: 
	QR:  1
	Opcode:  0000
	AA:  0
	TC:  0
	RD:  1
	RA:  1
	Z (Reserved):  000
	Rcode:  0000
Number of entries in question section:  1
Number of resource records in the answer section:  1
Number of name server resource records in the authority records section:  0
Number of resource records in the additional records section:  0


In [ ]:
canadian_answer_RR = parse_header(decoded_canadian_data)

ID:  1010101000000000
Flags:  1000000110000000
Flags Breakdown: 
	QR:  1
	Opcode:  0000
	AA:  0
	TC:  0
	RD:  1
	RA:  1
	Z (Reserved):  000
	Rcode:  0000
Number of entries in question section:  1
Number of resource records in the answer section:  4
Number of name server resource records in the authority records section:  0
Number of resource records in the additional records section:  0


## Step 4: Parse Response Message

In [137]:
def parse_answer(decoded_data):
    query = ""
    name = ""
    rr_type = ""
    class_rr_data = ""
    ttl = ""
    rdlength = ""
    rdata = ""
    start_index = 24 + jeff
    # Retrieve raw data
    for elementNAME in range(start_index, start_index+4):
        name = name + decoded_data[elementNAME]
    
    for elementRR_TYPE in range(start_index+4, start_index+8):
        rr_type = rr_type + decoded_data[elementRR_TYPE]
    
    for elementCLASS_RR_DATA in range(start_index+8, start_index+12):
        class_rr_data = class_rr_data + decoded_data[elementCLASS_RR_DATA]
    
    for elementTTL in range(start_index+12, start_index+20):
        ttl = ttl + decoded_data[elementTTL]
    
    for elementRDLENGTH in range(start_index+20, start_index+24):
        rdlength = rdlength + decoded_data[elementRDLENGTH]
    
    for elementRDATA in range(start_index+24, start_index+32):
        rdata = rdata + decoded_data[elementRDATA]
    
    # Begin post-processing
    address = int(rdata, 16)
    
    # Resource record class
    if class_rr_data == "0001":
        class_rr_data = "IN"
    
    # Resource record type
    if rr_type == "0001":
        rr_type = "A"
    elif rr_type == "0002":
        rr_type = "NS"

    # Offset for domain
    offset_val_raw = name[2:]
    offset_val = int(offset_val_raw, 16)
    offset_index = offset_val * 2
    #print("offset index: ", offset_index)
    
    # Get length of domain
    lengthOfDomain = ""
    for elementDomain in range(offset_index, offset_index+2):
        lengthOfDomain = lengthOfDomain + decoded_data[elementDomain]
    
    lengthOfDomain = int(lengthOfDomain)

    # Get hostname with length of domain
    hostname = ""
    for elementHOST in range(offset_index+2, offset_index+2+(lengthOfDomain*2)):
        hostname = hostname + decoded_data[elementHOST]
    
    # Get length of TLD
    tldStart = offset_index+2+(lengthOfDomain*2)
    lengthofTLD = ""
    for elementLENTLD in range(tldStart, tldStart+2):
        lengthofTLD = lengthofTLD + decoded_data[elementLENTLD]
    
    lengthofTLD = int(lengthofTLD)

    # Get TLD with length of TLD
    tld = ""
    for elementTLD in range(tldStart+2, tldStart+2+(lengthofTLD*2)):
        tld = tld + decoded_data[elementTLD]


    temp_host_name_object = bytes.fromhex(hostname)
    temp_tld_object = bytes.fromhex(tld)

    true_host_name = temp_host_name_object.decode("ASCII")
    true_tld = temp_tld_object.decode("ASCII")

    domain_name = true_host_name + "." + true_tld

    ip_address = socket.inet_ntoa(struct.pack(">L", address))

    print("Domain: ", domain_name)
    #print("DNS Record Type: ", rr_type)
    #print("Class: ", class_rr_data)
    #print("TTL: ", int(ttl, 16), "seconds")
    #print("RDLength: ", int(rdlength, 16))
    print("HTTP Server IP address: ", ip_address)

    return ip_address

In [ ]:
iranian_ip = parse_answer(decoded_iranian_data)

Domain:  tmz.com
HTTP Server IP address:  10.10.34.35


In [138]:
american_ip = parse_answer(decoded_american_data)

Domain:  facebook.com
HTTP Server IP address:  157.240.22.35


In [ ]:
canadian_ip = parse_answer(decoded_canadian_data)

Domain:  tmz.com
HTTP Server IP address:  18.65.229.44


## Step 5: Initiate TCP Connection

In [37]:
# Adapted from: https://www.geeks3d.com/hacklab/20190110/python-3-simple-http-request-with-the-socket-module/
# Iranian Request

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
start_time = time.time()
client.connect((iranian_ip, 80))

request = "GET / HTTP/1.1\r\nHost:tmz.com\r\n\r\n"
client.send(request.encode())

response = client.recv(4096)
print("Iranian HTTP RTT: " + str(time.time() - start_time))
http_response = repr(response)
with open("iranian_response_partA.html", "a") as f:
    f.write(http_response)



OSError: [Errno 113] No route to host

In [38]:
# American Request

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
start_time = time.time()
client.connect((american_ip, 80))

#request = b"GET tmz.com HTTP/1.1\n\n"
#request = "GET / HTTP/1.1\r\nHost:%s\r\n\r\n" % american_ip
request = "GET / HTTP/1.1\r\nHost:tmz.com\r\n\r\n"
client.send(request.encode())

response = client.recv(4096)
#print("American HTTP RTT: " + str(time.time() - start_time))
http_response = repr(response)
with open("american_response_partA.html", "a") as f:
    f.write(http_response)
    f.write("\n")


In [39]:
# Canadian Request

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
start_time = time.time()
client.connect((canadian_ip, 80))

#request = "GET / HTTP/1.1\r\nHost:%s\r\n\r\n" %canadian_ip
client.send(b"GET / HTTP/1.1\r\nHost:tmz.com\r\n\r\n")

response = client.recv(4096)
#print("Canadian HTTP RTT: " + str(time.time() - start_time))
http_response = repr(response)
with open("canadian_response_partA.html", "a") as f:
    f.write(http_response)
    f.write("\n")
